In [14]:
!pip install -qU "langchain[together]"



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [15]:
import os
from dotenv import load_dotenv

load_dotenv()


True

In [16]:
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")


In [17]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free", model_provider="together")


In [18]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, I'm Rayan")])


AIMessage(content='<think>\n\n</think>\n\nHi Rayan! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 9, 'total_tokens': 27, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b26e4043-d50a-4738-8d90-4f651e45f2ad-0', usage_metadata={'input_tokens': 9, 'output_tokens': 18, 'total_tokens': 27, 'input_token_details': {}, 'output_token_details': {}})

In [19]:
model.invoke([HumanMessage(content="What's my name ?")])


AIMessage(content="<think>\n\n</think>\n\nHi! I'm DeepSeek-R1, an AI assistant independently developed by the Chinese company DeepSeek Inc. For detailed information about models and products, please refer to the official documentation.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 8, 'total_tokens': 50, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f36c77a8-8602-44eb-ad98-fabb137d9fde-0', usage_metadata={'input_tokens': 8, 'output_tokens': 42, 'total_tokens': 50, 'input_token_details': {}, 'output_token_details': {}})

In [20]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi, I'm Rayan"),
    AIMessage(content="Hi Rayan! How can I assist you today?"),
    HumanMessage(content="What's my name ?")
])


AIMessage(content='<think>\nAlright, the user just asked, "What\'s my name?" Looking back at the conversation history, they introduced themselves as Rayan. So, they\'re probably testing if I remember or if I can recall previous information. I should confirm their name based on what they told me earlier. I\'ll respond by addressing them as Rayan and offer further assistance. Keeping it friendly and open-ended should encourage them to continue the conversation.\n</think>\n\nYour name is Rayan! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 28, 'total_tokens': 133, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c6a0fb6b-23c0-4a9b-9b1f-a8d2d1033a37-0', usage_metadata={'input_tokens': 28, 'output_tokens': 105, 'total_tokens': 133, 

In [29]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

#define a new graph
workflow = StateGraph(state_schema=MessagesState)

#define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

#define the single node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [30]:
config = {"configurable": {"thread_id": "abc123"}}


In [31]:
query = "Hi, I'm Rayan"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

<think>

</think>

Hi Rayan! How can I assist you today? 😊


In [ ]:
query = "What's my name ?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

<think>
Alright, the user just asked, "What's my name?" Looking back at the conversation history, they introduced themselves as Rayan. So, they're probably testing if I remember or if I can recall previous information. I should confirm their name based on what they told me earlier. I'll respond in a friendly and confident way to show that I remember. Maybe add an emoji to keep it warm and approachable.
</think>

Your name is Rayan! 😊


In [35]:
config = {"configurable": {"thread_id": "abc123"}}

query = "What's my name ?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================



Your name is Rayan! 😊


In [ ]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()
